# Introduction

In [1]:
#Import packages
import netCDF4
from netCDF4 import Dataset,num2date 
import numpy as np
import xarray as xr
import datetime
import pandas as pd
import matplotlib.pyplot as plt


# Download HYCOM netcdf

To download HYCOM data, open this link (https://tds.hycom.org/thredds/catalog.html) in your browser and navigate to the ocean model you want to use.

For Example: 
- scroll down to and select the "GLBu0.08/expt_19.1 (1995-08-01 to 2012-12-31)/" folder
- Then select the year "(2012) Reanalysis Data: 2012-01-01 to 2012-12-31 at 00Z"
- Select the OPENDAP access option
- Within this page, select the extent of the dimensions (lat.,long.,depth,time) and the extent of the variables (e.g. water_u, water_v, water_temp)
- Selecting and defining the extent of these dimensions and variables will adjust the 'Data URL' at the top of the page, which you will copy and paste to create the 'url' object in the script below

In [2]:
#Specify the URL taken from HYCOM...OPENDAP

url = 'https://tds.hycom.org/thredds/dodsC/GLBu0.08/expt_19.1/2012?depth[0:1:39],lat[1000:2000],lon[3000:4400],time[0:1:365],water_u[0:1:365][0:1:39][1000:2000][3000:4400],water_v[0:1:365][0:1:39][1000:2000][3000:4400],water_temp[0:1:365][0:1:39][1000:2000][3000:4400]'

# Download the data specified in the url as netcdf data
hycn = netCDF4.Dataset(url)

print(hycn)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    classification_level: UNCLASSIFIED
    distribution_statement: Approved for public release. Distribution unlimited.
    downgrade_date: not applicable
    classification_authority: not applicable
    institution: Naval Oceanographic Office
    source: HYCOM archive file
    history: archv2ncdf3z
    field_type: instantaneous
    Conventions: CF-1.0 NAVO_netcdf_v1.0
    dimensions(sizes): depth(40), lat(1001), lon(1401), time(366)
    variables(dimensions): float64 depth(depth), float64 lat(lat), float64 lon(lon), float64 time(time), int16 water_u(time, depth, lat, lon), int16 water_v(time, depth, lat, lon), int16 water_temp(time, depth, lat, lon)
    groups: 


## Explore HYCOM data

In [3]:
# Save hycom netcdf as x array to be manipulated and saved
hycx = xr.open_dataset(url) # If the netcdf does not already have a crs, the code for lat long is "epsg:4326"

    # View xarray object 
hycx

<xarray.Dataset>
Dimensions:     (depth: 40, lat: 1001, lon: 1401, time: 366)
Coordinates:
  * depth       (depth) float64 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * lat         (lat) float64 0.0 0.08 0.16 0.24 0.32 ... 79.76 79.84 79.92 80.0
  * lon         (lon) float64 60.0 60.08 60.16 60.24 ... 171.8 171.8 171.9 172.0
  * time        (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-12-31
Data variables:
    water_u     (time, depth, lat, lon) float32 ...
    water_v     (time, depth, lat, lon) float32 ...
    water_temp  (time, depth, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Naval Oceanographic Office
    source:                    HYCOM archive file
    history:                   archv2ncdf3z
    field_type:                instantaneous
    Conventions:               CF-1.0 NAVO_netcdf_v1.0

## Exporting HYCOM data foruse with PARCELS

PARCELS requires a file directory (not just a url to create a fieldset within which particles are dispersed

In [4]:
#Save just the study region (Northwest Pacific)
nwp = hycx.sel(lat=slice(10, 38), 
    lon=slice(118, 150))

#Save just the surface of the study region
nwp_surf = hycx.sel(lat=slice(10, 38), 
    lon=slice(118, 150), depth=0)

#Save a slice at 900m
nwp_900 = hycx.sel(lat=slice(10, 38), 
    lon=slice(118, 150), depth=9.00e+02)

#Save a slice at 1000m
nwp_1000 = hycx.sel(lat=slice(10, 38), 
    lon=slice(118, 150), depth=1.00e+03)

#nwp_surf.to_netcdf("HYCOM/nwp_surf.nc")
#nwp_900.to_netcdf("HYCOM/nwp_900.nc")
nwp_1000.to_netcdf("HYCOM/nwp_1000.nc")


At this point you can navigate to your newly created netcdf file in Terminal and use command $ncview 'filename' to view the netcdf file as an interactive map

### File Structure

Many of the PARCEL tutorials have the netcdf files parsed by time, if necessary, this can be achieved using the code below

In [5]:
# Export nwp_surf as individual day time slices

days, datasets = zip(*nwp_surf.groupby("time"))

timestamps = np.arange('2012-01', '2013-01', dtype='datetime64[D]')

paths = [f"HYCOM/nwp_surf_{y}.nc" for y in timestamps]


#xr.save_mfdataset(datasets, paths)

In [6]:
# Export nwp_1000 as individual day time slices

days, datasets = zip(*nwp_1000.groupby("time"))

timestamps = np.arange('2012-01', '2013-01', dtype='datetime64[D]')

paths = [f"HYCOM/nwp_1000_{y}.nc" for y in timestamps]


xr.save_mfdataset(datasets, paths)